(1) Numeric Example: KV(prefix) is invariant to suffix

We will use a tiny causal transformer with:

	•	vocab → toy integers
	•	embedding size → 2
	•	1 attention layer
	•	Q/K/V are linear projections

We show how token #1 and #2 KV are the same, even if the suffix changes.

Prompt A: Tell me a joke.

Tokens: [10, 20, 30, 40]

Prompt B: Tell me a riddle
Tokens: [10, 20, 30, 99]

The prefix is: [10, 20, 30]

Embeddings:

E(10) = [1, 0]

E(20) = [0, 1]

E(30) = [1, 1]

E(40) = [2, 0]

E(99) = [0, 2]

Linear Projections

Wk = [[1, 0], [0, 1]]   -> identity

Wv = [[1, 0], [0, 1]]   -> identity

Compute K/V for prefix:

Token 10

K10 = Wk * [1,0] = [1,0]

V10 = Wv * [1,0] = [1,0]

Token 20

K20 = [0,1]

V20 = [0,1]

Token 30

K30 = [1,1]

V30 = [1,1]

These K/V values do not depend on tokens 40 or 99.

So regardless of the suffix:

Prompt A (10,20,30,40) produces:

Prefix K = [[1,0], [0,1], [1,1]]

Prefix V = [[1,0], [0,1], [1,1]]

Prompt B (10,20,30,99) produces:

Prefix K = [[1,0], [0,1], [1,1]]

Prefix V = [[1,0], [0,1], [1,1]]

Because the projections depend only on prefix embeddings, not on future tokens.

This generalizes exactly to real transformers with many layers:
K/V for token t depend only on tokens 1…t.

How vLLM Detects Shared Prefixes (Exact Token-Level Match)

vLLM implements prefix caching using a structure called PagedAttention which stores KV in “virtual memory pages.”

How vLLM matches prefixes across sequences

vLLM:

	1.	Tokenizes the input
	2.	Stores tokens in a “KV page table”
	3.	Compares new token sequences with existing cached ones
	4.	Finds the longest common prefix
	5.	Reuses all KV pages corresponding to that prefix
	6.	Computes only the new suffix tokens
	7.	Appends KV pages for the new tokens

Example:

Cached request:

[User: Tell me a]

New request

[User: Tell me a joke]

Common prefix = [User:, Tell, me, a]

New tokens = [joke]

It then:

	•	Reuses all KV pages for prefix
	•	Only runs the model for “joke”
